In [1]:
import datetime
import pandas as pd
import time
import requests
import os
import numpy as np

In [2]:
DATA_DIR = "data/djia_20000101_20171101/"


In [4]:
class Hedge(object):
    
    def __init__(self, directory, wealth):
        self.directory = directory
        
        self.assets = [f.split(".")[0].upper() for f in os.listdir(directory)]
        self.data_iters = [pd.read_csv(directory+s.lower()+".csv", iterator=True, chunksize=1) for s in self.assets]
        
        self.cursors = [d.get_chunk(1) for d in self.data_iters]
        self.prices = np.asarray([float(c["adj_close"]) for c in self.cursors])
        
        self.benchmark_weights = np.ones(len(self.assets))/len(self.assets)
        self.weights = np.ones(len(self.assets))/len(self.assets)
        
        self.position = [weight * wealth for weight in self.weights]
        self.benchmark_position = [weight * wealth for weight in self.benchmark_weights]
        
        self.benchmark_wealth = np.sum(self.benchmark_position)
        self.wealth = np.sum(self.position)
        
        self.returns = None
        self.eta = -0.0001
        
    def move_cursor_and_update(self):
        self.cursors = [d.get_chunk(1) for d in self.data_iters]
        new_prices = np.asarray([float(c["adj_close"]) for c in self.cursors])
        self.returns = (new_prices - self.prices)/self.prices
        self.prices = new_prices
        return
        
    def calc_weights(self):       
        multipliers = np.exp(self.eta * self.returns/np.sum(self.weights * self.returns))
        self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers) 
        return
    
    def adjust_position(self):
        self.position = np.asarray([weight * self.wealth for weight in self.weights]) * (1 + self.returns)
        self.wealth = np.sum(self.position)
        self.benchmark_position = np.asarray([weight * self.benchmark_wealth for weight in self.benchmark_weights]) * (1 + self.returns)
        self.benchmark_wealth = np.sum(self.benchmark_position)
        return
        
    def run(self):
        while True:
            try:
                self.move_cursor_and_update()
                self.calc_weights()
                self.adjust_position()
            except StopIteration:
                break

In [5]:
h = Hedge(DATA_DIR, 10000)
h.run()

print h.wealth
print h.benchmark_wealth

19057.6841678
19647.7308454


In [6]:
h.assets

['AAPL',
 'AXP',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'DD',
 'DIS',
 'GE',
 'GS',
 'HD',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PFE',
 'PG',
 'TRV',
 'UNH',
 'UTX',
 'V',
 'VZ',
 'WMT',
 'XOM']